In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import open3d as o3d


from objects_clustering import *
from tqdm import trange

In [ ]:

file_path = 'C:/Users/Hp/Desktop/CathayHackathon/Cathay_pcd/point_cloud.ply'

pcd = o3d.io.read_point_cloud(file_path)



point_data, color_data = np.asarray(pcd.points, dtype=object), np.asarray(pcd.colors, dtype=object)


In [ ]:
def o3d_single_vis(points, colors):

    pcd_data = o3d.geometry.PointCloud()

    pcd_data.points = o3d.utility.Vector3dVector(points)
    pcd_data.colors = o3d.utility.Vector3dVector(colors)

    vis = o3d.visualization.Visualizer()
    vis.create_window()
    vis.add_geometry(pcd_data)
    vis.run()
    vis.destroy_window()

def o3d_cluster_vis(points, colors, random_color_flag = True):

    pcd_data = o3d.geometry.PointCloud()
    random_colors = np.random.rand(len(points), 3)
    
    for i in trange(len(points)):
        single_pcd_data = o3d.geometry.PointCloud()
        single_pcd_data.points = o3d.utility.Vector3dVector(points[i])
        if random_color_flag == True:
            single_pcd_data.paint_uniform_color(random_colors[i])
        else:
            single_pcd_data.colors = o3d.utility.Vector3dVector(colors[i])

        pcd_data += single_pcd_data

    vis = o3d.visualization.Visualizer()
    vis.create_window()
    vis.add_geometry(pcd_data)
    vis.run()
    vis.destroy_window()

@njit
def get_points_from_indices(point_data, indices):
    return point_data[np.asarray(indices)]

In [ ]:
o3d_single_vis(point_data, color_data)

In [ ]:
grid_data = Initialize_Grid(point_data = point_data, unit_size = [0.1, 0.1, 0.1])

grid, points_to_grid_dict, grid_to_points_dict, valid_bottom_tiles_by_xy, stacked_bottom_tiles_to_points_by_xy = grid_data

In [ ]:
# point_data = np.random.rand(10000000, 3)

cluster_indices = object_cluster(point_data, unit_size = [0.1, 0.1, 0.05])

cluster_points = [get_points_from_indices(point_data, i) for i in cluster_indices]
cluster_colors = [get_points_from_indices(color_data, i) for i in cluster_indices]

o3d_cluster_vis(cluster_points, cluster_colors, random_color_flag = False)

In [ ]:
file_num = 0
for cluster, color in zip(cluster_points, cluster_colors):

    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(np.asarray(cluster))
    point_cloud.colors = o3d.utility.Vector3dVector(np.asarray(color))

    # o3d.io.write_point_cloud('luggage'+str(file_num)+'.ply',point_cloud)



## save clusters